 Import the libraries

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import WordPunctTokenizer
import os
import urllib.request
import tensorflow as tf
import torch
import os
import torchvision
import tarfile
from torch.utils.data import random_split
from torchvision.datasets.utils import download_url
import matplotlib.pyplot as plt
import torch.nn as nn
import spacy
from torch.utils.data import DataLoader

In [2]:
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

from transformers.models.bert.modeling_bert import BertPooler, BertSelfAttention
from transformers import BertTokenizer
from torch.utils.data import Dataset
#pretrained_bert="bert-base-uncased"
import json

In [3]:
#for getting the package versions used in this notebook

import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system names
        if name == "PIL":
            name = "Pillow"
        elif name == "sklearn":
            name = "scikit-learn"

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

In [4]:

#function to get the device being used.use CUDA for inferencing

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [5]:

#utility class for training and testing.Needs to be present in all notebooks since it is inherited by models.


class aspectClassificationBase(nn.Module):

    def training_step(self,batch):#batch wise training
        model.train()
        text,aspect,labels=batch
        out=self(text,aspect)
        #labels=int(labels)
        labels=labels.to(torch.long)
        loss=F.cross_entropy(out,labels)
        return loss
    
    def validation_step(self,batch):#batch wise validation
        model.eval()
        text,aspect,labels=batch
        out=self(text,aspect)
        labels=labels.to(torch.long)
        loss=F.cross_entropy(out,labels)
        acc=accuracy(labels,out)
        return {'val_loss':loss.detach(),'val_acc':acc}
    
    def validation_epoch_end(self,result):
        loss=[x['val_loss'] for x in result]
        acc= [x['val_acc'] for x in result]
        l_b=torch.stack(loss).mean()
        a_b=torch.stack(acc).mean()
        return {'val_loss':l_b.item(), 'val_acc': a_b.item()}
    
    def epoch_end(self,epoch,result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch,result['val_loss'], result['val_acc']))

In [6]:
class LSTM_imp(aspectClassificationBase):#model class for BI-LSTM
    def __init__(self,emb_mat,num_classes,embedding_dim,vocab_size_t,vocab_size_a):
        super(LSTM_imp,self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size_t = vocab_size_t
        self.vocab_size_a=vocab_size_a
        self.polarities=num_classes
        self.embed = nn.Embedding.from_pretrained(emb_mat,freeze=True)
        self.lstm =nn.LSTM(self.embedding_dim,128,batch_first=True,num_layers=2,dropout=0.2,bidirectional=True)
        #self.lstm_a  =nn.LSTM(self.embedding_dim,128,batch_first=True,bidirectional=True)
        self.dense = nn.Linear(self.vocab_size_t*self.vocab_size_a,self.polarities)
        #self.softmax=nn.Softmax(dim=1)

    def forward(self, text,aspect):
        #asp_raw_indices=inputs[:,278:,:]
       # y1=inputs[0][278:][:]
        #y2=inputs[1][278:][:]
        
        #text_raw_indices = torch.stack([x1,x2])
        #asp_raw_indices=torch.stack([y1,y2])
        #print(asp_raw_indices.shape)
        #print(text_raw_indices.shape)
        x= self.embed(text)
        x,_=self.lstm(x)
        #print(x.shape)
        
        x2=self.embed(aspect)
        x2,_=self.lstm(x2)
        #print(x2.shape)
        x2=x2.permute(0,2,1)
        
        x3=torch.matmul(x,x2)
        #print(x3.shape)
        x3=x3.reshape(batch_size,-1)
        x3=self.dense(x3)
        #print(x3.shape)
       
        #x_len = torch.sum(text_raw_indices != 0, dim=-1)
        #_, (h_n, _) = self.lstm(x, x_len)
        #out = self.dense(h_n[0])
        return x3

In [7]:
class LSTM_AOA(aspectClassificationBase):#model class for Attention over attention
    def __init__(self,emb_mat,num_classes,embedding_dim,vocab_size_t,vocab_size_a):#emb_mat stores glove embeddings of all 
                                                                                   #words in our dictionary
        super(LSTM_AOA,self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size_t = vocab_size_t
        self.vocab_size_a=vocab_size_a
        self.polarities=num_classes
        self.embed = nn.Embedding.from_pretrained(emb_mat,freeze=True)#use glove word embedding matrix 
        self.lstm =nn.LSTM(self.embedding_dim,128,batch_first=True,num_layers=1,bidirectional=True)
        #self.lstm_a  =nn.LSTM(self.embedding_dim,128,batch_first=True,bidirectional=True)
        self.dense = nn.Linear(256,self.polarities)
        self.softmax_c=nn.Softmax(dim=2)
        self.softmax_r=nn.Softmax(dim=1)
        self.softmax=nn.Softmax(dim=1)
        
    def forward(self, text,aspect,batch_size):
        #asp_raw_indices=inputs[:,278:,:]
       # y1=inputs[0][278:][:]
        #y2=inputs[1][278:][:]
        
        #text_raw_indices = torch.stack([x1,x2])
        #asp_raw_indices=torch.stack([y1,y2])
        #print(asp_raw_indices.shape)
        #print(text_raw_indices.shape)
        x= self.embed(text)
        x,_=self.lstm(x)
       # print(x.shape)
        
        x2=self.embed(aspect)
        x2,_=self.lstm(x2)
        #print(x2.shape)
        x2=x2.permute(0,2,1)
        
        x3=torch.matmul(x,x2)
        #print(x3.shape)
        x4=x3.detach().clone()
        x5=x3.detach().clone()
        x5=self.softmax_c(x5)
        #print(x5.shape)
        x4=self.softmax_r(x4)
        #x4=x4.permute(0,2,1)
        x5=x5.sum(dim=1)/self.vocab_size_t
        #print(x5.shape)
        #print(x5)
        x5=x5.reshape(batch_size,self.vocab_size_a,1)
        x6=torch.matmul(x4,x5)
     
       # print(x6.shape)
       # print(x6.shape)
        x=x.permute(0,2,1)
       # print(x.shape)
        x7=torch.matmul(x,x6)
        #print(x7.shape)
        
        
        x7=x7.reshape(batch_size,-1)
        #print(x7.shape)
        x7=self.dense(x7)#no need for softmax if cross entropy loss is being used as a loss function.
        #print(x7.shape)
        #out=self.softmax(x7)
        #x_len = torch.sum(text_raw_indices != 0, dim=-1)
        #_, (h_n, _) = self.lstm(x, x_len)
        #out = self.dense(h_n[0])
        return x7

In [ ]:
device=get_default_device()#get available device

# **LSTM_AOA is a bit more accurate than LSTM_imp**
AOA-66.8% 
LSTM_imp-62%

In [94]:
model_name= input("Enter model")
print(model_name)

In [95]:
if model_name=='LSTM_AOA':
    b=20
elif model_name=='LSTM_imp':
    b=2

In [96]:
if b==20:
    model = torch.load('../input/d/enkrish259/aoa-colwise-softmax/aoa_b20_e30_entire_model.pth')#load AOA_Lstm  model
elif b==2:
    model=torch.load('../input/lstm-vocab/lstm_bi_e30.pth')#load lstm_imp model
to_device(model,device)
model.eval()

In [18]:
#vocab

In [16]:
#vocab

In [14]:
#idx2word

In [97]:
#text encoding/vocab building
def inf_prep(data):
    vocab_file= open("../input/lstm-vocab/full_vocabulary.json")
    vocab_f=vocab_file.read()
    vocab=json.loads(vocab_f)#load json file which stores words to index dictionary for our data
    if type(data)==float:
        data=str(data)
    text2=data.lower()
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(text2)
    words = [word for word in tokens if word.isalnum()]
    w_ind={v: int(k) for k, v in vocab.items()}
   # print(w_ind)
    encoded_sentence = np.array([w_ind[word] for word in words])#word to index embedded sentence
    return encoded_sentence
    
    

In [98]:
#encoded_text

In [125]:
#function in which model inference occurs
def infer(test_dl,model,b):
    with torch.no_grad():
        for batch in test_dl:
            text,aspect=batch
           # print(text)
            #print(aspect)
            out=model(text,aspect,b)#use this for AOA
            #out=model(text,aspect) -use this one for LSTM_IMP
            
            _, preds = torch.max(out, dim=1)
            #print(preds)
            preds=preds.cpu().detach().numpy()
            preds=preds.tolist()
            return {'label':preds[0]}

In [128]:
def predict_ans(text,aspect,b,device,model):
    tex=np.empty([b,278])
    aspec=np.empty([b,8])
    encoded_text=inf_prep(text)
    encoded_text=np.append(encoded_text,np.zeros(278-len(encoded_text)))
    encoded_aspect=inf_prep(aspect)
    encoded_aspect=np.append(encoded_aspect,np.zeros(8-len(encoded_aspect)))

    for i in range(b):
        tex[i]=encoded_text
        aspec[i]=encoded_aspect
    tex=torch.LongTensor(tex)
    aspec=torch.LongTensor(aspec)
    test_ds=TensorDataset(tex,aspec)
    #input batch size=20 for LSTM_AOA and 2 for LSTM_imp
    test_dl = DataLoader(test_ds, b, shuffle=False)
    test_dl=DeviceDataLoader(test_dl,device)
    ans=infer(test_dl,model,b)
    if ans['label']==0:
        print('Negative sentiment')
    elif ans['label']==1:
        print('Neutral sentiment')
    elif ans['label']==2:
        print('Positive sentiment')
    return ans

    

In [ ]:
text = input("Enter your text/context: ")
print(text)

In [ ]:
aspect= input("Enter aspect ")
print(aspect)


In [129]:
predict_ans(text,aspect,b,device,model)